In [ ]:
import sympy as sp

a11, a12 = sp.symbols('a11 a12', real=True)
a21, a22 = sp.symbols('a21 a22', real=True)
A = sp.Matrix(
    [
        [a11, a12],
        [a21, a22],
    ]
)

orth = (A.T @ A - sp.eye(2)).norm(ord='fro') ** 2

avec = sp.Matrix([a11, a12, a21, a22])



In [ ]:
orth

In [ ]:
grad = sp.Matrix([orth.diff(x) for x in avec])
grad

In [ ]:
hess = sp.Matrix(
    [
        [grad[i].diff(avec[j]) for j in range(4)]
        for i in range(4)
    ]
)
hess

In [ ]:
hess.inv()

In [ ]:
import sympy as sp
from sympy.utilities.codegen import codegen
def gen_rust(fn_name, expr):
    return codegen(
        (fn_name, expr),
        language="Rust",
        prefix="rust_code"
    )[0][1]

In [ ]:
for i, g in enumerate(grad):
    print(gen_rust(f"grad_{i}", g))
    print()

In [ ]:
for i in range(hess.shape[0]):
    for j in range(hess.shape[1]):
        h = hess[i, j]
        print(gen_rust(f"affine_hess_{i}_{j}", h))
        print()

## Contact
node -> affine

In [ ]:
import sympy as sp

from sympy.utilities.codegen import codegen
def gen_rust(fn_name, expr):
    return codegen(
        (fn_name, expr),
        language="Rust",
        prefix="rust_code"
    )[0][1]

# initial position
px, py = sp.symbols('px py', real=True)
ux, uy, vx, vy = sp.symbols('ux uy vx vy', real=True)

# 定义向量
u = sp.Matrix([ux, uy])
v = sp.Matrix([vx, vy])
p = sp.Matrix([px, py])

a11, a12 = sp.symbols('a11 a12', real=True)
a21, a22 = sp.symbols('a21 a22', real=True)
tx, ty = sp.symbols('tx ty', real=True)
A = sp.Matrix(
    [
        [a11, a12],
        [a21, a22],
    ]
)
t = sp.Matrix([tx, ty])

x = A @ p + t

q = sp.Matrix([tx, ty, a11, a12, a21, a22])


## case 1

In [ ]:
u_v = v - u
u_x = x - u

# if uv.dot(ux) <= 0
dist = u_x.norm()
grad1 = dist.diff(q)
grad1

In [ ]:
for i in range(6):
    print(gen_rust(f"grad_case1_{i}", grad1[i]))

In [ ]:
hess1 = sp.Matrix.zeros(len(q), len(q))
for i in range(len(q)):
    for j in range(len(q)):
        hess1[i, j] = dist.diff(q[i]).diff(q[j])

hess1


In [ ]:
for i in range(6):
    for j in range(6):
        print(gen_rust(f"hess_case1_{i}_{j}", hess1[i, j]))

case 2

In [ ]:
import sympy as sp

from sympy.utilities.codegen import codegen
def gen_rust(fn_name, expr):
    return codegen(
        (fn_name, expr),
        language="Rust",
        prefix="rust_code"
    )[0][1]

# initial position
px, py = sp.symbols('px py', real=True)
ux, uy, vx, vy = sp.symbols('ux uy vx vy', real=True)

# 定义向量
u = sp.Matrix([ux, uy])
v = sp.Matrix([vx, vy])
p = sp.Matrix([px, py])

a11, a12 = sp.symbols('a11 a12', real=True)
a21, a22 = sp.symbols('a21 a22', real=True)
tx, ty = sp.symbols('tx ty', real=True)
A = sp.Matrix(
    [
        [a11, a12],
        [a21, a22],
    ]
)
t = sp.Matrix([tx, ty])

x = A @ p + t

q = sp.Matrix([tx, ty, a11, a12, a21, a22])


################## compute ##################
dist = (x - v).norm()
hess2 = sp.Matrix.zeros(len(q), len(q))
for i in range(len(q)):
    for j in range(len(q)):
        hess2[i, j] = dist.diff(q[i]).diff(q[j])

hess2

In [ ]:
for i in range(6):
    for j in range(6):
        print(gen_rust(f"hess_case2_{i}_{j}", hess2[i, j]))

case3

In [ ]:
uv = v - u
ux = x - u

dist = abs(sp.Matrix([[uv, ux]]).det()) / uv.norm()

grad3 = dist.diff(q)

In [ ]:
for i in range(6):
    print(gen_rust(f"grad_case3_{i}", grad3[i]))

In [6]:
import sympy as sp
from sympy.utilities.codegen import codegen

# 定义符号
px, py = sp.symbols('px py', real=True)
ux, uy, vx, vy = sp.symbols('ux uy vx vy', real=True)
tx, ty = sp.symbols('tx ty', real=True)
a11, a12, a21, a22 = sp.symbols('a11 a12 a21 a22', real=True)

# 定义向量
u = sp.Matrix([ux, uy])
v = sp.Matrix([vx, vy])
p = sp.Matrix([px, py])
A = sp.Matrix([[a11, a12], [a21, a22]])
t = sp.Matrix([tx, ty])

# 计算点 x
x = A @ p + t

# 计算直线 uv 的方向向量和法向量
d = v - u
n = sp.Matrix([-d[1], d[0]])

# 计算点 x 到直线 uv 的距离
distance = (n.dot(x - u)) / n.norm()

# 定义 q
q = sp.Matrix([tx, ty, a11, a12, a21, a22])

# 计算距离对 q 的梯度
grad_distance = sp.Matrix([sp.diff(distance, q_i) for q_i in q])

# 计算 Hessian 矩阵
hessian_distance = sp.Matrix([[sp.diff(grad_distance[i], q_j) for q_j in q] for i in range(len(q))])

hessian_distance

Matrix([
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0]])

In [ ]:
for i in range(6):
    for j in range(6):
        print(gen_rust(f"hess_case3_{i}_{j}", hess3[i, j]))

In [7]:
x

Matrix([
[a11*px + a12*py + tx],
[a21*px + a22*py + ty]])

In [ ]:
sp.DiracDelta()